In [ ]:
### Raízes chebyshev #####
function raizes_cheby(ordem)
    m = ordem + 1
    raizes = -cos.((2*LinRange(1,m,m) .-1)*pi/(2*m))
    return raizes
end

In [ ]:
## polinomio de chebyshev não recursivo 
function poli_cheb(ordem;capital)
    d = ordem
    t = zeros(length(capital),d+1)
    for j in 0:d
        t[:,j + 1] = cos.(j*acos.(capital))
    end
    return t
end

In [ ]:
function consumo(gamas;teis)
    c = teis*gamas
    return c
end

In [ ]:
#################################
## tudo isso na função residuo ##
#################################
function res_pc(gamas,d,grid_z,P)
    nz = length(grid_z)
    res = zeros(d+1,nz)

    r_1 = raizes_cheby(d) # raízes para calcular o polinomio
    k_0 = cb_ss(r_1) # capital em nível

    t = poli_cheb(d; capital = r_1) # termos do polinomio

    c_0 = consumo(gamas; teis = t) # consumo com polinomios

    k_1 = k_0.^(1/3)*grid_z' .+ (1 - 0.012).*k_0 .- c_0
    capital_1 = cb_zero(k_1)  # normalizamos p/ [-1,1]

    for estado in 1:nz
        teis_1 = poli_cheb(d; capital = capital_1[:,estado]) # polinomios novos
        c_1 = consumo(gamas;teis = teis_1)
        ulinha = c_1.^(-2)
        dcdk = (1/3)*k_1[:,estado].^(-2/3)*grid_z' .+ (1-0.012) 
        lde = ulinha.*dcdk
        for id in 1:d+1
            res[id,estado] = c_0[id,estado]^(-2) - 0.987*dot(P[estado,:],lde[id,:])
        end
    end
    return res
end


In [ ]:
function gama_otimo_pc(d,grid_z,P)
    nz = length(grid_z)
    guess = ones(2,nz)
    s=1
    gamaotimo = Array
    while s <= 5
        g(gamas) = res_pc(gamas,s,S,P)
        gamaotimo = nlsolve(g,guess).zero
        guess = vcat(gamaotimo, zeros(1,nz))
        s = s+1
    end
    return gamaotimo
end

In [ ]:
function policy_pc(d,grid_k,grid_z,P)
    r_1 = cb_zero(grid_k)
    t = poli_cheb(d; capital = r_1) # termos do polinomio

    c_0 = consumo(gama_otimo_pc(d,grid_z,P); teis = t) # consumo com polinomios

    k_1 = grid_k.^(1/3)*grid_z' .+ (1 - 0.012).*grid_k .- c_0

    return c_0, k_1
end

In [ ]:
##########################
#### erros de Euler ######
function euler_error_pc(gamas,d,grid_k,grid_z,P)
    nz = length(grid_z)
    nk = length(grid_k)
    r_1 = cb_zero(grid_k)
    t = poli_cheb(d; capital = r_1) # termos do polinomio

    c_0 = consumo(gamas; teis = t) # consumo com polinomios

    k_1 = grid_k.^(1/3)*grid_z' .+ (1 - 0.012).*grid_k .- c_0
    capital_1 = cb_zero(k_1)  # normalizamos p/ [-1,1]

    for estado in 1:nz
        teis_1 = poli_cheb(d; capital = capital_1[:,estado]) # polinomios novos
        c_1 = consumo(gamas;teis = teis_1)
        ulinha = c_1.^(-2)
        dcdk = (1/3)*k_1[:,estado].^(-2/3)*grid_z' .+ (1-0.012) 
        lde = ulinha.*dcdk
        for id in 1:nk
             inversa = (0.987*dot(P[estado,:],lde[id,:]))^(-1/2)
             eee[id,estado] = 1 - inversa/c_0[id,estado]
        end
    end
    eee = log10.(abs.(eee))
    return eee
end
